In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 此處需要登入 google 帳號

import os

os.chdir('/content/drive/My Drive/MyResearch') #切換該目錄

Mounted at /content/drive


In [ ]:
!pip install pykalman

     |████████████████████████████████| 228 kB 5.3 MB/s 
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48462 sha256=ed155275dac2fbb8fd7b782cf854e492a9eac62afd498e238eb7020429b6aead
  Stored in directory: /root/.cache/pip/wheels/6a/04/02/2dda6ea59c66d9e685affc8af3a31ad3a5d87b7311689efce6
Successfully built pykalman


In [ ]:
import math
import csv
import os
import time
from itertools import combinations, permutations
import statistics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import pydot
from pykalman import KalmanFilter
from scipy.spatial.distance import cosine
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import export_graphviz
from statsmodels.tsa.api import ExponentialSmoothing, Holt, SimpleExpSmoothing
from scipy.signal import savgol_filter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
allzero = "./20220510無線訊號/Wireless/allzero.csv"

sniffer0 = "./20220510無線訊號/Wireless/sniffer_zero.csv"
sniffer1 = "./20220510無線訊號/Wireless/sniffer_one.csv"
sniffer2 = "./20220510無線訊號/Wireless/sniffer_two.csv"
sniffer3 = "./20220510無線訊號/Wireless/sniffer_three.csv"
sniffer5 = "./20220510無線訊號/Wireless/sniffer_five.csv"

sniffer_list = [sniffer0,sniffer1,sniffer2,sniffer3,sniffer5]
# file_name_list_test = [sniffer0, sniffer1]

alpha = 0.5

start_time = '20220510 14:51:30'
end_time = '20220510 15:11:30'
start_time = start_time[9:len(start_time)]
end_time = end_time[9:len(end_time)]

mac_queue=['0000ce9c-0000-1000-8000-00805f9b34fb', # u ultra
       '000027ef-0000-1000-8000-00805f9b34fb', # u11
       '0000b1fc-0000-1000-8000-00805f9b34fb', # V4025
       '0000f22e-0000-1000-8000-00805f9b34fb', # u19e
       '0000dd9b-0000-1000-8000-00805f9b34fb', # V4445
       '0000950b-0000-1000-8000-00805f9b34fb', # MotoG
       '00007dd1-0000-1000-8000-00805f9b34fb'] # a51

# dev_queue=['dev1','dev2','dev3','dev4','dev5','dev6','dev7']
dev_queue=['UUltra','U11','V4025','U19e','V4445','MotoG','A51']

In [ ]:
### RSSI正規化
def getNormalize(column):
    column = ( column - column.min() ) / ( column.max() - column.min() ) * 100
    return column

In [ ]:
### 簡易指數平滑，降噪用
def getSES(probe, alpha=None, auto=True):
    # # print(type(probe.mean()))
    if math.isnan(probe.mean()):
        # print(probe.mean())
        ses=pd.read_csv(allzero, sep=',', parse_dates=[0], error_bad_lines=False, warn_bad_lines=False,encoding='ISO-8859-15')
        ses = ses.set_index('time')
        ses.RSSI.loc[ses.RSSI!=0]=0
        print(ses)
        return ses
    fit = SimpleExpSmoothing(probe).fit(smoothing_level=alpha, optimized=auto)
    ses = fit.fittedvalues.shift(-1)
    ses = pd.DataFrame(ses).reset_index()
    ses.columns = ['time','RSSI']
    ses = ses.set_index('time')
    # ses.to_csv('allZero.log')
    return ses

In [ ]:
def getRawData(filename, device_mac):
  df = pd.read_csv(filename, parse_dates=[0])
  df.columns = ['time','mac','type','RSSI','uuid']
  rate='1'
  df = df.set_index('time')
  df = df.between_time(start_time, end_time)
  df = df[(df.mac == device_mac)|(df.uuid == device_mac)].resample(rate+'S').agg(dict(RSSI='max'))
  # 正規化
  df.RSSI = getNormalize(df.RSSI).fillna(0)
  # 平滑化
  SES = getSES(df.RSSI, alpha, False).dropna()
  return SES

In [ ]:
### 將不同sniffer中關於同一手機資料蒐集在一起
def merge_All_Sniffer_Data_about_one_Phone(sniffer_list, mac_name):
    mergedata = getRawData(sniffer_list[0], mac_name)
    for sniffer in sniffer_list[1:] :
        mergedata = pd.merge(mergedata,getRawData(sniffer,mac_name),on='time',how='outer').sort_values(by='time').fillna(0)
    mergedata.columns = ['sniffer0','sniffer1','sniffer2','sniffer3','sniffer5']
    return mergedata

In [ ]:
def Inner_Product(first, second):
    ans = 0
    for i in range(5):
        ans = ans + first[i] * second[i]
    return ans

In [ ]:
### 計算2台phone之間的pairwise mobility relaionship
def getPairwiseMobilityRelationship(first_user, second_user):
    ans = 0
    min_between_input = min(len(second_user.index), len(first_user.index))
    # print(first_user, second_user)
    for i in range(min_between_input):
        ans = ans + Inner_Product(first_user.iloc[i], second_user.iloc[i])
    
    return ans / min_between_input / 5 / 5

In [ ]:
spatio_temporal_feature_pattern = []
mac_queue_test = [mac_queue[5], mac_queue[6]]
for mac in mac_queue_test:
    spatio_temporal_feature_pattern.append(merge_All_Sniffer_Data_about_one_Phone(sniffer_list, mac))
# print(spatio_temporal_feature_pattern)
C = getPairwiseMobilityRelationship(spatio_temporal_feature_pattern[0], spatio_temporal_feature_pattern[1])
print(C)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'RSSI_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.


62.41523487269395


In [ ]:
counter=len(mac_queue)
for i in range(len(mac_queue)-1):
  for j in range(1,counter):
    mac_queue_test = [mac_queue[i], mac_queue[i+j]]
    spatio_temporal_feature_pattern = []
    for mac in mac_queue_test:
        spatio_temporal_feature_pattern.append(merge_All_Sniffer_Data_about_one_Phone(sniffer_list, mac))
    # print(spatio_temporal_feature_pattern)
    C = getPairwiseMobilityRelationship(spatio_temporal_feature_pattern[0], spatio_temporal_feature_pattern[1])
    print(dev_queue[i],dev_queue[i+j],C)
  counter-=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'RSSI_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.


UUltra U11 22.179332571385785
UUltra V4025 31.609449612359544
UUltra U19e 33.93070880461174
UUltra V4445 113.46806008299946
UUltra MotoG 74.80833281518348
UUltra A51 48.659544157224296
U11 V4025 35.59634456008145
U11 U19e 84.76371127862973
U11 V4445 44.603988878819145
U11 MotoG 26.459918916264108
U11 A51 16.287316530218888
V4025 U19e 56.25828384945987
V4025 V4445 65.83607178323956
V4025 MotoG 37.52875466062252
V4025 A51 24.43039653777684
U19e V4445 66.64238551959204
U19e MotoG 36.708745042310895
U19e A51 23.031061743701486
V4445 MotoG 130.93117151165734
V4445 A51 85.13100569823737
MotoG A51 62.41523487269395
